# Vector

In [20]:
class Ord v => Vector v where
    distance :: v -> v -> Double
    centroid :: [v] -> v

In [2]:
-- for declaration of type variables
{-# LANGUAGE FlexibleInstances #-}

In [42]:
instance Vector (Double, Double) where
    distance (a,b) (c,d) = sqrt $ (c-a)*(c-a) + (d-b)*(d-b)
    centroid lst =
        let (u,v) = foldr (\(a,b) (c,d) -> (a+c, b+d)) (0,0) lst
            n = fromIntegral $ length lst
        in (u / n, v / n)

In [43]:
{-# LANGUAGE MultiParamTypeClasses #-}

In [5]:
class Vector v => Vectorizable e v where
    toVector :: e -> v

instance Vectorizable (Double, Double) (Double, Double) where
    toVector = id

# KMeans

In [6]:
kMeans :: (Vector v, Vectorizable e v)
        => (Int -> [e] -> [v]) -- number of centroids and initialization
        -> [e] -- the information
        -> [v] -- centroids after convergence

: 

※型宣言のみなのでエラーが出ている

In [7]:
import Data.List
import qualified Data.Map as M

In [9]:
:t minimumBy

minimumBy :: forall (t :: * -> *) a. Foldable t => (a -> a -> Ordering) -> t a -> a

In [11]:
:t compare

compare :: forall a. Ord a => a -> a -> Ordering

In [12]:
compare 1 2

LT

In [13]:
minimumBy compare [1,2,3]

Line 1: Use minimum
Found:
minimumBy compare
Why not:
minimum

1

In [17]:
:t M.adjust

M.adjust :: forall k a. Ord k => (a -> a) -> k -> Map k a -> Map k a

In [19]:
clusterAssignmentPhase :: (Ord v, Vector v, Vectorizable e v)
                        => [v] -> [e] -> M.Map v [e]
clusterAssignmentPhase centroids points =
    let initialMap = M.fromList $ zip centroids (repeat [])
        in foldr (\p m -> let chosenC = minimumBy (compareDistance p) centroids
                            in M.adjust (p:) chosenC m) initialMap points
    where compareDistance p x y = compare (distance x $ toVector p) (distance y $ toVector p)

In [46]:
newCentroidPhase :: (Vector v, Vectorizable e v) => M.Map v [e] -> [(v,v)]
newCentroidPhase = M.toList . fmap (centroid . map toVector)

In [47]:
shouldStop :: (Vector v) => [(v,v)] -> Double -> Bool
shouldStop centroids threshold =
    foldr (\(x,y) s -> s + distance x y) 0.0 centroids < threshold

In [72]:
kMeans' :: (Vector v, Vectorizable e v)
        => [v] -> [e] -> Double -> [v]
kMeans' centroids points threshold =
    let assignments = clusterAssignmentPhase centroids points
        oldNewCentroids = newCentroidPhase assignments
        newCentroids = map snd oldNewCentroids
    in if shouldStop oldNewCentroids threshold
       then newCentroids
       else kMeans' newCentroids points threshold
      
kMeans :: (Vector v, Vectorizable e v)
        => (Int -> [e] -> [v])  -- initialization function
        -> Int -- number of centroids
        -> [e] -- the information
        -> Double -- threshold
        -> [v] -- final centroid
kMeans i k points = kMeans' (i k points) points

In [74]:
initializeSimple :: Int -> [e] -> [(Double, Double)]
initializeSimple 0 _ = []
initializeSimple n v = (fromIntegral n, fromIntegral n) : initializeSimple (n-1) v

In [75]:
let info = [(1,1), (1,2), (4,4), (4,5)]::[(Double,Double)]

In [76]:
kMeans initializeSimple 2 info 0.001

[(1.0,1.5),(4.0,4.5)]